<a href="https://colab.research.google.com/github/prasann25/colab/blob/main/09_milestone_project_2_skimlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Milestone Project 2: SkimLit

The purpose of this notebook is to build an NL P model to to make reading medical abstracts easier.
The paper we're replicating (the source of the dataset that we'll be using) is available here : https://arxiv.org/abs/1710.06071

And reading through the paper above, we see that the model architecture that they use to achieve their best results is available here:
https://arxiv.org/abs/1612.05251

**Resources** If you want to find the ground truth for this notebook(with lots of diagrams and text annotations) see the GitHub :
https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/09_SkimLit_nlp_milestone_project_2.ipynb

## Confirm access to GPU


In [4]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-d0b6ac4b-9c31-c9cd-31be-a0eaa79ba074)


## Get data
Since we'll be replicating the paper above(PubMed 200K RCT), let's download the dataset they use.
We can do so from the authors GitHub :
https://github.com/Franck-Dernoncourt/pubmed-rct


In [5]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct
!ls pubmed-rct

Cloning into 'pubmed-rct'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 33 (delta 0), reused 0 (delta 0), pack-reused 30
Unpacking objects: 100% (33/33), done.
PubMed_200k_RCT
PubMed_200k_RCT_numbers_replaced_with_at_sign
PubMed_20k_RCT
PubMed_20k_RCT_numbers_replaced_with_at_sign
README.md


In [6]:
# Check what files are in the PubMed_20K dataset
! ls pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/
#! ls pubmed-rct/PubMed_20k_RCT/

dev.txt  test.txt  train.txt


In [7]:
# Start our experiments using the 20K dataset with numbers replaced by "@" sign
data_dir = "/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/"


In [8]:
# Check all of the filenames in the target directory 
import os
filenames = [data_dir + filename for filename in os.listdir(data_dir)]
filenames

['/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/dev.txt',
 '/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/train.txt',
 '/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/test.txt']

## Preprocess data
Now we've got some text data, it's time to become one with it.
And one of the best ways to become one with the data is to....

> Visualize, Visualize, Visualize

So with that in mind, let's write a function to read all the lines of a target text file.

In [9]:
# Create function to read the lines of a document

def get_lines(filename) :
  """
  Reads filename(a text filename) and returns the lines of text as a list.
  
  Args:
    filename: a string containing the target filepath.

  Returns :
  A list of strings with one string per line from the target filename.
  """
  with open(filename, "r") as f :
    return f.readlines()


In [10]:
# Let's read in the training lines
train_lines = get_lines(data_dir  + "train.txt")
train_lines[:28]

['###24293578\n',
 'OBJECTIVE\tTo investigate the efficacy of @ weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at @ weeks in older adults with moderate to severe knee osteoarthritis ( OA ) .\n',
 'METHODS\tA total of @ patients with primary knee OA were randomized @:@ ; @ received @ mg/day of prednisolone and @ received placebo for @ weeks .\n',
 'METHODS\tOutcome measures included pain reduction and improvement in function scores and systemic inflammation markers .\n',
 'METHODS\tPain was assessed using the visual analog pain scale ( @-@ mm ) .\n',
 'METHODS\tSecondary outcome measures included the Western Ontario and McMaster Universities Osteoarthritis Index scores , patient global assessment ( PGA ) of the severity of knee OA , and @-min walk distance ( @MWD ) .\n',
 'METHODS\tSerum levels of interleukin @ ( IL-@ ) , IL-@ , tumor necrosis factor ( TNF ) - , and 

In [11]:
len(train_lines)

210040

Let's think about how we want our data to look...

How I think our data would be best represented ...

```
[{'line_number' : 0,
   'target' : 'BACKGROUND',
   'text' : "Emotional eating is associated with overeating and the development of obesity .\n",
   'total_lines' : 11},
   ...]   
```

Let's write a function which turns each of our datasets into the above format, so we can continue to prepare our data for modelling.


In [14]:
def preprocess_text_with_line_numbers(filename) :
  """
  Returns list of dictionary of abstract line data.

  Takes in filename, reads it contents and sort through each line, 
  extracting things like the target label, text of the sentence,
  how many sentences are in the current abstract and what sentence number
  the target line is
  """
  input_lines = get_lines(filename) # get all lines from filename
  abstract_lines = "" # create an empty abstract
  abstract_samples = [] # create an empty list of abstracts

  # Loop through each line in the target line
  for line in input_lines :

    if line.startswith("###") : # check to see if the line is an ID line
      abstract_id = line
      abstract_lines= "" # reset the abstract string if the line is an ID line

    elif line.isspace() : # check to see if the line is a new line
      abstract_line_split = abstract_lines.splitlines() # split abstract lines into separate lines     
      # Iterate through each line in a single abstract and count at same time
      for abstract_line_number, abstract_line in enumerate(abstract_line_split) :
        target_text_split = abstract_line.split("\t") # split target label from text
        line_data ={} # create an empty dictionar for each line
        line_data["target"] = target_text_split[0]
        line_data["text"] = target_text_split[1].lower() # get target text and lower it
        line_data["line_number"] = abstract_line_number # what number line does the line appear on the abstract
        line_data["total_lines"] = len(abstract_line_split) -1 # how many total lines are there in the target abstract? (start from 0) 
        abstract_samples.append(line_data) # add line data to abstract samples list

    else : # if the above conditions aren't fulfilled, the line contains a labelled sentence
      abstract_lines += line

  return abstract_samples



In [17]:
# Get data from file and preprocess it
%%time
train_samples = preprocess_text_with_line_numbers(data_dir + "train.txt")
val_samples = preprocess_text_with_line_numbers(data_dir + "dev.txt") # dev is another name for val
test_samples = preprocess_text_with_line_numbers(data_dir + "test.txt")
print(len(train_samples), len(val_samples), len(test_samples))

180040 30212 30135
CPU times: user 441 ms, sys: 89.6 ms, total: 530 ms
Wall time: 533 ms


In [18]:
# Check the first abstract of our training  data
train_samples[:14]

[{'line_number': 0,
  'target': 'OBJECTIVE',
  'text': 'to investigate the efficacy of @ weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at @ weeks in older adults with moderate to severe knee osteoarthritis ( oa ) .',
  'total_lines': 11},
 {'line_number': 1,
  'target': 'METHODS',
  'text': 'a total of @ patients with primary knee oa were randomized @:@ ; @ received @ mg/day of prednisolone and @ received placebo for @ weeks .',
  'total_lines': 11},
 {'line_number': 2,
  'target': 'METHODS',
  'text': 'outcome measures included pain reduction and improvement in function scores and systemic inflammation markers .',
  'total_lines': 11},
 {'line_number': 3,
  'target': 'METHODS',
  'text': 'pain was assessed using the visual analog pain scale ( @-@ mm ) .',
  'total_lines': 11},
 {'line_number': 4,
  'target': 'METHODS',
  'text': 'secondary outcome measures include